# JupyterLabにおけるGUI作成の概要

JupyterLabは、データ解析や機械学習、科学計算の分野で広く利用されている対話型開発環境です。JupyterLabは、コード、グラフ、Markdown文書などを統合的に扱える柔軟なワークスペースを提供し、Pythonをはじめとする多くの言語カーネルをサポートしています。さらに、JupyterLabでは、**GUI** をNotebook内に直接組み込むことができ、対話的なアプリケーションを簡単に構築することができます。

ここでは、代表的な2つのGUI構築ライブラリである **ipywidgets** と **Panel** を用いた例を紹介します。

# ipywidgetsによるGUI作成

`ipywidgets` は、Jupyter NotebookやJupyterLabで使用できる公式のウィジェットライブラリです。スライダーやボタン、ドロップダウンメニューなどのインタラクティブなコントロールを作成でき、ユーザー操作に応じてPythonコードを動的に実行できます。

以下の例では、`ipywidgets`を用いて**ローレンツ方程式（Lorenz system）**のシミュレーションを可視化するGUIを作成します。ユーザーはスライダーでパラメータを調整し、3次元プロット上で軌道の変化を確認できます。

In [10]:
import numpy as np
from scipy import integrate
import plotly.graph_objects as go
from ipywidgets import FloatSlider, IntSlider, VBox, interactive_output
import ipywidgets as widgets

# ローレンツ方程式を数値的に解く関数
def solve_lorenz(N=10, angle=0.0, max_time=4.0, sigma=10.0, beta=8./3, rho=28.0):
    def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
        x, y, z = x_y_z
        return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

    np.random.seed(1)
    x0 = -15 + 30 * np.random.random((N, 3))
    t = np.linspace(0, max_time, int(250 * max_time))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0i, t) for x0i in x0])
    return t, x_t

# 3D図形ウィジェットを作成
fig = go.FigureWidget()
fig.add_trace(go.Scatter3d(mode='lines', line=dict(width=1)))

# スライダー操作時に呼び出される描画更新関数
def update_plot(angle=0.0, max_time=4.0, N=10, sigma=10.0, rho=28.0):
    t, x_t = solve_lorenz(N=N, angle=angle, max_time=max_time, sigma=sigma, rho=rho)
    current_traces = len(fig.data)

    with fig.batch_update():
        # 既存の軌道を更新
        for i in range(min(current_traces, N)):
            x, y, z = x_t[i, :, :].T
            fig.data[i].x = x
            fig.data[i].y = y
            fig.data[i].z = z

        # 新しい軌道を追加
        for i in range(current_traces, N):
            x, y, z = x_t[i, :, :].T
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(width=1)))

        # 余分な軌道を削除
        if len(fig.data) > N:
            fig.data = fig.data[:N]
            
        # カメラ角度とレイアウトを更新
        fig.update_layout(
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                camera=dict(eye=dict(
                    x=np.cos(np.radians(angle)), 
                    y=np.sin(np.radians(angle)), 
                    z=0.5
                ))
            ),
            width=400, height=400,
            title="Lorenz System Simulation"
        )

# スライダーを作成
slider_layout = widgets.Layout(width='350px')

angle_slider = FloatSlider(value=0.0, min=0.0, max=360.0, step=1.0, description='angle',
                           layout=slider_layout, continuous_update=False)
max_time_slider = FloatSlider(value=4.0, min=0.1, max=4.0, step=0.1, description='max_time',
                              layout=slider_layout, continuous_update=False)
N_slider = IntSlider(value=10, min=0, max=50, step=1, description='N',
                     layout=slider_layout, continuous_update=False)
sigma_slider = FloatSlider(value=10.0, min=0.0, max=50.0, step=0.5, description='sigma',
                           layout=slider_layout, continuous_update=False)
rho_slider = FloatSlider(value=28.0, min=0.0, max=50.0, step=0.5, description='rho',
                         layout=slider_layout, continuous_update=False)

# ウィジェットレイアウトと描画の連携
ui = VBox([angle_slider, max_time_slider, N_slider, sigma_slider, rho_slider, fig])
out = interactive_output(update_plot, {
    'angle': angle_slider,
    'max_time': max_time_slider,
    'N': N_slider,
    'sigma': sigma_slider,
    'rho': rho_slider
})
ui

このコードでは、`interactive_output()`を用いて、スライダーの値が変更された際に`update_plot()`関数が呼び出され、`plotly`の3Dグラフが動的に更新されます。`FigureWidget`を利用することで、`Plotly`と`ipywidgets`を緊密に連携させることが可能です。

---

# PanelによるGUI作成

`Panel`は、`Bokeh`を基盤とした高度なダッシュボード・アプリケーション作成ライブラリです。`ipywidgets`よりも柔軟なレイアウト管理が可能であり、Notebook内だけでなくWebアプリケーションとしても動作します。

以下は、同様のローレンツ方程式を`Panel`で実装した例です。

In [11]:
import numpy as np
from scipy import integrate
import plotly.graph_objects as go
import panel as pn

pn.extension('plotly')

def solve_lorenz(N=10, angle=0.0, max_time=4.0, sigma=10.0, beta=8./3, rho=28.0):
    def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
        x, y, z = x_y_z
        return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

    np.random.seed(1)
    x0 = -15 + 30 * np.random.random((N, 3))
    t = np.linspace(0, max_time, int(250 * max_time))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0i, t) for x0i in x0])
    return t, x_t

def create_figure(angle=0.0, max_time=4.0, N=10, sigma=10.0, rho=28.0):
    t, x_t = solve_lorenz(N, angle, max_time, sigma, rho)
    fig = go.Figure()
    for i in range(N):
        x, y, z = x_t[i, :, :].T
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(width=1)))
    fig.update_layout(
        scene=dict(
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
            camera=dict(eye=dict(
                x=np.cos(np.radians(angle)), 
                y=np.sin(np.radians(angle)), 
                z=0.5
            ))
        ),
        width=500, height=500,
        title="Lorenz System Simulation"
    )
    return fig

# パネルウィジェットを定義
angle = pn.widgets.FloatSlider(name='angle', start=0, end=360, step=1, value=0)
max_time = pn.widgets.FloatSlider(name='max_time', start=0.1, end=4.0, step=0.1, value=4.0)
N = pn.widgets.IntSlider(name='N', start=0, end=50, step=1, value=10)
sigma = pn.widgets.FloatSlider(name='sigma', start=0, end=50, step=0.5, value=10.0)
rho = pn.widgets.FloatSlider(name='rho', start=0, end=50, step=0.5, value=28.0)

# バインディングでリアクティブ更新
interactive_plot = pn.bind(create_figure, angle, max_time, N, sigma, rho)

# レイアウトを構築
dashboard = pn.Row(
    pn.Column(angle, max_time, N, sigma, rho),
    pn.panel(interactive_plot)
)

dashboard

Row
    [0] Column
        [0] FloatSlider(end=360, name='angle', step=1)
        [1] FloatSlider(end=4.0, name='max_time', start=0.1, value=4.0)
        [2] IntSlider(end=50, name='N', value=10)
        [3] FloatSlider(end=50, name='sigma', step=0.5, value=10.0)
        [4] FloatSlider(end=50, name='rho', step=0.5, value=28.0)
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)

`Panel`では`pn.bind()`関数を用いることで、ウィジェットの値とプロット関数を直接バインドできます。Notebook上では即座に反映され、さらに`dashboard.show()`を呼び出せば、同じインターフェースをブラウザ上の独立したWebアプリとして表示することも可能です。

---

# まとめ

`ipywidgets`と`Panel`はいずれもJupyterLab上で強力なGUI構築手段を提供します。  
- `ipywidgets`は軽量でNotebookとの親和性が高く、教育・実験向きです。  
- `Panel`はレイアウトや配信機能が豊富で、研究成果を共有するWebアプリ開発に適しています。

これらを活用することで、コードと可視化を統合した対話的なデータ解析環境を容易に構築することができます。